In [70]:
import pandas as pd
import numpy as np
import time
from pandas import Timestamp
import datetime

In [71]:
df_X = pd.read_csv(f'dataset/solar_汙水廠(history).csv', header=None)
dfXlen = len(df_X)
#從df_X中選取所有列的資料[從第一列開始,篩選到DataFrame物件的最後一列,保留DataFrame物件中所有的列]
df_X = df_X.iloc[0:dfXlen,:]
df_X = df_X.drop(columns=[5, 9, 13, 14, 20, 21, 22])#刪除IBM欄位
df_X

,0,1,2,3,4,6,7,8,10,11,12,15,16,17,18,19,23,24,25
0,TIME_TO_INTERVAL,Power,Radiation,ClearSkyRadiation,Radiation(SDv3)(CWB),Radiation(SDv3)(OWM),Radiation(MSM),Radiation(today)(CWB),Radiation(today)(OWM),WeatherType(CWB),WeatherType(pred)(CWB),WeatherType(OWM),WeatherType(pred)(OWM),ApparentTemperature(pred)[CWB],Temperature(pred)[CWB],RelativeHumidity(pred)[CWB],FeelsLikeTemperature(pred)[OWM],Temperature(pred)[OWM],RelativeHumidity(pred)[OWM]
1,2021-04-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.720000000000034,23.460000000000036,71.0
2,2021-04-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.720000000000034,23.460000000000036,71.0
3,2021-04-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.720000000000034,23.460000000000036,71.0
4,2021-04-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.94,23.590000000000032,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17888,2023-08-08 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.02000000000004,32.81,57.0
17889,2023-08-08 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.02000000000004,32.81,57.0
17890,2023-08-08 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.700000000000045,33.36000000000001,58.0
17891,2023-08-08 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.700000000000045,33.36000000000001,58.0


# <font color=#0000FF>取大表(solar_汙水廠(history))進行4倍擴增</font>

In [72]:
#整個時間欄(col)
time_all = df_X[0].values

df_X1 = df_X.iloc[0,:].to_frame().T

for j in range(1,dfXlen):
    #每隔15分鐘產生一個時間，共產生4個
    df_datatime = pd.date_range(start=time_all[j], periods=4, freq='15min')
    #第j行(讀取每一個row)
    df_X_original =df_X.loc[j,:]

    for i in range(1,4):
        #將就的替換成新的 ex:2020-06-01 20:00:00 -> 2020-06-01 20:15:00
        df_X_new = df_X_original.replace(str(df_datatime[0]), str(df_datatime[i]))
        #將需要做線性的欄位值都設為空,以利後續做concat
        df_X_new[[1,2,3,4,6,7,8,10,17,18,19,23,24,25]] = np.nan
    
        if i==1 and j ==1:
            #to_frame().T為轉置
            df_X1 = pd.concat([df_X,df_X_new.to_frame().T],axis=0, ignore_index=True)
        else:
            df_X1 = pd.concat([df_X1,df_X_new.to_frame().T],axis=0, ignore_index=True)
        
# df_X1 = df_X1.sort_values(by=0)
df_X1.to_csv(f'dataset/solar_汙水廠_newbig(history).csv', index=None,header=False)

# <font color=#0000FF>將solar_汙水廠_newbig(history)檔依時間排序</font>

In [73]:
df_X = pd.read_csv(f'dataset/solar_汙水廠_newbig(history).csv') 
df_X = df_X.sort_values(by='TIME_TO_INTERVAL')
df_X.to_csv(f'dataset/solar_汙水廠_newbig_sort(history).csv', index=None)

# <font color=#0000FF>讀取15分鐘csv檔(該檔尚未做線性)</font>

In [74]:
df_X = pd.read_csv(f'./dataset/solar_汙水廠_newbig_sort(history).csv')

# <font color=#0000FF>轉DataFrame做線性補值</font>

In [75]:
# 創建數據集
df_TIME_TO_INTERVAL =pd.DataFrame(df_X['TIME_TO_INTERVAL'])
df_X1 = pd. DataFrame({
  'Power': df_X['Power'].astype(float),
  'Radiation': df_X['Radiation'].astype(float),
  'ClearSkyRadiation': df_X['ClearSkyRadiation'].astype(float),
  'Radiation(SDv3)(CWB)': df_X['Radiation(SDv3)(CWB)'].astype(float),
  'Radiation(SDv3)(OWM)': df_X['Radiation(SDv3)(OWM)'].astype(float),
  'Radiation(MSM)': df_X['Radiation(MSM)'].astype(float),
  'Radiation(today)(CWB)': df_X['Radiation(today)(CWB)'].astype(float),
  'Radiation(today)(OWM)': df_X['Radiation(today)(OWM)'].astype(float),
})
df_X2 = pd. DataFrame({
  'WeatherType(CWB)': df_X['WeatherType(CWB)'],
  'WeatherType(pred)(CWB)': df_X['WeatherType(pred)(CWB)'],
  'WeatherType(OWM)': df_X['WeatherType(OWM)'],
  'WeatherType(pred)(OWM)': df_X['WeatherType(pred)(OWM)']  
})
df_X3 = pd. DataFrame({
  'ApparentTemperature(pred)[CWB]': df_X['ApparentTemperature(pred)[CWB]'].astype(float),
  'Temperature(pred)[CWB]': df_X['Temperature(pred)[CWB]'].astype(float),
  'RelativeHumidity(pred)[CWB]': df_X['RelativeHumidity(pred)[CWB]'].astype(float),
  'FeelsLikeTemperature(pred)[OWM]': df_X['FeelsLikeTemperature(pred)[OWM]'].astype(float),
  'Temperature(pred)[OWM]': df_X['Temperature(pred)[OWM]'].astype(float),
  'RelativeHumidity(pred)[OWM]': df_X['RelativeHumidity(pred)[OWM]'].astype(float)  
})

# 線性進行插值(備注:df_X2為名目資料,不需要做線性補值)
df_X1 = df_X1.interpolate(method ='linear',limit_area='inside')#limit_area='inside'僅填充被有效值包圍的 NaN（插值）
df_X3 = df_X3.interpolate(method ='linear',limit_area='inside')

#合併資料欄
df_all = pd.concat([df_TIME_TO_INTERVAL,df_X1,df_X2,df_X3],axis=1)
df_all
#存成csv
df_all.to_csv(f'./dataset/solar_汙水廠_newbig_sort(history_15m).csv', index=None)

# <font color=#0000FF>讀取solar_汙水廠(new).csv擴增</font>

In [1]:
def Auto_merge_15m():
    import os
    import pandas as pd
    #取現在時間
    now = datetime.datetime.now() 
    #讀檔
    solar_new = pd.read_csv(f'dataset/solar_汙水廠(new).csv', header=None)
    dfXlen = len(solar_new)
    solar_new = solar_new.iloc[0:dfXlen,:]
    solar_new = solar_new.drop(columns=[5, 9, 13, 14, 20, 21, 22])#刪除IBM欄位

    #取solar_汙水廠(new)的TIME_TO_INTERVAL欄值(col)
    # solar_new_noheader格式=2023/1/26 00:00
    solar_new_noheader = solar_new[0]
    #將 solar_new_noheader 中的TIME_TO_INTERVAL資料解析為 datetime 物件且從第一筆開始取資料(不含TIME_TO_INTERVAL標題欄)
    solar_new_noheader[1:] = pd.to_datetime(solar_new_noheader[1:], format='%Y/%m/%d %H:%M')
    # solar_new_noheader格式= 2023-01-26 00:00:00
    solar_new_noheader = solar_new_noheader[1:]

    # 取代原本的TIME_TO_INTERVAL欄的值(因solar_new_noheader沒有TIME_TO_INTERVAL標題欄)
    solar_new_noheader.loc[0] ='TIME_TO_INTERVAL'
    solar_new[0] = solar_new_noheader

    #取solar_汙水廠(new)的第一筆TIME_TO_INTERVAL
    new_first_col = solar_new[0]
    new_first_row = new_first_col[1]
    #因new_first_row是Timestamp,需轉時間字串
    new_first_row = new_first_row.strftime('%Y-%m-%d %H:%M:%S')

    #創建空的dataframe
    solar_new_dataframe = [
        {'TIME_TO_INTERVAL'},
        {'Power'},
        {'Radiation'},
        {'ClearSkyRadiation'},
        {'Radiation(SDv3)(CWB)'},
        {'Radiation(SDv3)(OWM)'},
        {'Radiation(MSM)'},
        {'Radiation(today)(CWB)'},
        {'Radiation(today)(OWM)'},
        {'WeatherType(CWB)'},
        {'WeatherType(pred)(CWB)'},
        {'WeatherType(OWM)'},
        {'WeatherType(pred)(OWM)'},
        {'ApparentTemperature(pred)[CWB]'},
        {'Temperature(pred)[CWB]'},
        {'RelativeHumidity(pred)[CWB]'},
        {'FeelsLikeTemperature(pred)[OWM]'},
        {'Temperature(pred)[OWM]'},
        {'RelativeHumidity(pred)[OWM]'}
    ]

    # 將 solar_new_dataframe 中的元素轉換為列表
    cols = [list(x)[0] for x in solar_new_dataframe]

    # 創建空的 dataframe
    solar_new_dataframe = pd.DataFrame(columns=cols)

    #整個TIME_TO_INTERVAL時間欄(col)
    time_all = solar_new.iloc[1:,0]  

    for j in range(0,dfXlen-1):
        #每隔15分鐘產生一個時間，共產生4個
        time_all_new = time_all.iloc[j]
        time_str = time_all_new.strftime('%Y-%m-%d %H:%M:%S')
        #df_datatime[0]=00:00,df_datatime[1]=00:15,df_datatime[2]=00:30,df_datatime[3]=00:45
        df_datatime = pd.date_range(start=time_str, periods=4, freq='15min')
        #第j+1行(一小時的),j欄是標題欄
        solar_new_original =solar_new.loc[j+1,:]
        #第一筆位置為0~3,第一筆後要從位置3之後開始存資料
        if j == 0:
            j = j   
        else:
            j = j+(j*3)

        # 添加第一行數據(EX:第一筆)
        solar_new_dataframe.loc[j, 'TIME_TO_INTERVAL'] = df_datatime[0]
        solar_new_dataframe.loc[j, 'Power'] = solar_new_original[1]
        solar_new_dataframe.loc[j, 'Radiation'] = solar_new_original[2]
        solar_new_dataframe.loc[j, 'ClearSkyRadiation'] = solar_new_original[3]
        solar_new_dataframe.loc[j, 'Radiation(SDv3)(CWB)'] = solar_new_original[4]
        solar_new_dataframe.loc[j, 'Radiation(SDv3)(OWM)'] = solar_new_original[6]
        solar_new_dataframe.loc[j, 'Radiation(MSM)'] = solar_new_original[7]
        solar_new_dataframe.loc[j, 'Radiation(today)(CWB)'] = solar_new_original[8]
        solar_new_dataframe.loc[j, 'Radiation(today)(OWM)'] = solar_new_original[10]
        solar_new_dataframe.loc[j, 'WeatherType(CWB)'] = solar_new_original[11]
        solar_new_dataframe.loc[j, 'WeatherType(pred)(CWB)'] =solar_new_original[12]
        solar_new_dataframe.loc[j, 'WeatherType(OWM)'] = solar_new_original[15]
        solar_new_dataframe.loc[j, 'WeatherType(pred)(OWM)'] = solar_new_original[16]
        solar_new_dataframe.loc[j, 'ApparentTemperature(pred)[CWB]'] =solar_new_original[17]
        solar_new_dataframe.loc[j, 'Temperature(pred)[CWB]'] = solar_new_original[18]
        solar_new_dataframe.loc[j, 'RelativeHumidity(pred)[CWB]'] = solar_new_original[19]
        solar_new_dataframe.loc[j, 'FeelsLikeTemperature(pred)[OWM]'] =solar_new_original[23]
        solar_new_dataframe.loc[j, 'Temperature(pred)[OWM]'] = solar_new_original[24]
        solar_new_dataframe.loc[j, 'RelativeHumidity(pred)[OWM]'] =solar_new_original[25]
        # 添加第2到4行數據(00:15~00:45)(EX:第二到四筆)
        for s in range(1, 4):
            solar_new_dataframe.loc[j+s, 'TIME_TO_INTERVAL'] = df_datatime[s]
            #天氣類型為名目資料,因此填入與一小時資料相同類型
            solar_new_dataframe.loc[j+s, 'WeatherType(CWB)'] = solar_new_original[11]
            solar_new_dataframe.loc[j+s, 'WeatherType(pred)(CWB)'] = solar_new_original[12]
            solar_new_dataframe.loc[j+s, 'WeatherType(OWM)'] = solar_new_original[15]
            solar_new_dataframe.loc[j+s, 'WeatherType(pred)(OWM)'] = solar_new_original[16]


    # 轉DataFrame做線性補值
    df_TIME_TO_INTERVAL = pd. DataFrame({
      'TIME_TO_INTERVAL': solar_new_dataframe['TIME_TO_INTERVAL']
    })
    df_X1 = pd. DataFrame({
      'Power': solar_new_dataframe['Power'].astype(float),
      'Radiation': solar_new_dataframe['Radiation'].astype(float),
      'ClearSkyRadiation': solar_new_dataframe['ClearSkyRadiation'].astype(float),
      'Radiation(SDv3)(CWB)': solar_new_dataframe['Radiation(SDv3)(CWB)'].astype(float),
      'Radiation(SDv3)(OWM)': solar_new_dataframe['Radiation(SDv3)(OWM)'].astype(float),
      'Radiation(MSM)': solar_new_dataframe['Radiation(MSM)'].astype(float),
      'Radiation(today)(CWB)': solar_new_dataframe['Radiation(today)(CWB)'].astype(float),
      'Radiation(today)(OWM)': solar_new_dataframe['Radiation(today)(OWM)'].astype(float),
    })
    df_X2 = pd. DataFrame({
      'WeatherType(CWB)': solar_new_dataframe['WeatherType(CWB)'],
      'WeatherType(pred)(CWB)': solar_new_dataframe['WeatherType(pred)(CWB)'],
      'WeatherType(OWM)': solar_new_dataframe['WeatherType(OWM)'],
      'WeatherType(pred)(OWM)': solar_new_dataframe['WeatherType(pred)(OWM)']  
    })
    df_X3 = pd. DataFrame({
      'ApparentTemperature(pred)[CWB]': solar_new_dataframe['ApparentTemperature(pred)[CWB]'].astype(float),
      'Temperature(pred)[CWB]': solar_new_dataframe['Temperature(pred)[CWB]'].astype(float),
      'RelativeHumidity(pred)[CWB]': solar_new_dataframe['RelativeHumidity(pred)[CWB]'].astype(float),
      'FeelsLikeTemperature(pred)[OWM]': solar_new_dataframe['FeelsLikeTemperature(pred)[OWM]'].astype(float),
      'Temperature(pred)[OWM]': solar_new_dataframe['Temperature(pred)[OWM]'].astype(float),
      'RelativeHumidity(pred)[OWM]': solar_new_dataframe['RelativeHumidity(pred)[OWM]'].astype(float)  
    })
    # 線性進行插值，對所有NaN值進行填充
    # 線性進行插值，對所有NaN值進行填充
    df_X1 = df_X1.interpolate(method='linear', limit_area=None)
    df_X3 = df_X3.interpolate(method='linear', limit_area=None)


    # 合併資料欄
    df_all = pd.concat([df_TIME_TO_INTERVAL, df_X1, df_X2, df_X3], axis=1)

    # 刪除舊的檔案
    file_path = './dataset/solar_汙水廠_newbig_sort(realtime_history_15m).csv'
    if os.path.exists(file_path):
        os.remove(file_path)
        print('舊的檔案已刪除')

    # 將新的15分鐘資料存到CSV檔案中
    df_all.to_csv(f'./dataset/solar_汙水廠_newbig_sort(realtime_history_15m).csv', mode='a', index=False)
    print(now,'新的15分鐘資料存到CSV檔案中')
    #----------------------------------------------------------------------------------

    # 將Power的值替換成15分鐘(原始直接轉15分鐘)
    # 讀取兩個 CSV 檔案

    #一小時轉15分鐘(Power是一小時轉15m的)
    df_newbig = pd.read_csv('./dataset/solar_汙水廠_newbig_sort(realtime_history_15m).csv')

    #Power是原始直接轉15分鐘(資料來源:get_power_15)
    df_history = pd.read_csv('./power_data/solar_汙水廠_history(get_15m_power).csv')

    # 將 'TIME_TO_INTERVAL' 欄位設定為日期時間格式
    df_newbig['TIME_TO_INTERVAL'] = pd.to_datetime(df_newbig['TIME_TO_INTERVAL'])
    df_history['TIME_TO_INTERVAL'] = pd.to_datetime(df_history['TIME_TO_INTERVAL'])

    # 合併兩個 DataFrame，根據 'TIME_TO_INTERVAL' 欄位進行內連接
    df_merged = pd.merge(df_newbig, df_history[['TIME_TO_INTERVAL', 'Power']], on='TIME_TO_INTERVAL', how='left')

    # 將 'Power' 欄位的值從 'df_history' 取代 'df_newbig' 的值，即使 'df_newbig' 的值是空值
    df_merged['Power'] = df_merged['Power_y'].combine_first(df_merged['Power_x'])

    # 移除多餘的欄位
    df_merged = df_merged[['TIME_TO_INTERVAL', 'Power', 'Radiation', 'ClearSkyRadiation', 'Radiation(SDv3)(CWB)', 'Radiation(SDv3)(OWM)', 'Radiation(MSM)', 'Radiation(today)(CWB)', 'Radiation(today)(OWM)', 'WeatherType(CWB)', 'WeatherType(pred)(CWB)', 'WeatherType(OWM)', 'WeatherType(pred)(OWM)', 'ApparentTemperature(pred)[CWB]', 'Temperature(pred)[CWB]', 'RelativeHumidity(pred)[CWB]', 'FeelsLikeTemperature(pred)[OWM]', 'Temperature(pred)[OWM]', 'RelativeHumidity(pred)[OWM]']]

    # 儲存結果到新的 CSV 檔案
    df_merged.to_csv('./dataset/solar_汙水廠_merged(15m_ch_power).csv', index=False)
    print(now,'Power的值替換成15分鐘')
    Forecast_day_by_4_15_minutes()

# 即時預測當日下四個逐15分鐘發電量

In [2]:
def split_data(data):
    import pandas as pd
    feature_data = pd.DataFrame(data,columns=['up_Power_1','up_Power_2','up_Radiation_1','now_Radiation','next_Radiation_1'])
    length_data = int(len(data))
    #next_Radiation_1(倒數4筆沒有下一小時的Radiation)
    length_data_last = length_data-4
    for j in range(0,length_data):
        #up_Power_1和up_Radiation_1
        #(開頭4筆沒有上一小時的Radiation和Power)
        if j in range(0,4):
            feature_data['up_Power_1'][j] = 0.0000
            feature_data['up_Radiation_1'][j] = 0.0000
        else:
            feature_data['up_Power_1'][j] = data['Power'][j-4]
            feature_data['up_Radiation_1'][j] = data['Radiation(today)(CWB)'][j-4]
        
        #up_Power_2
        #(開頭8筆沒有上兩小時的Radiation和Power)
        if j in range(0,8):
            feature_data['up_Power_2'][j] = 0.0000
        else:
            feature_data['up_Power_2'][j] = data['Power'][j-8]
        
        #now_Radiation
        feature_data['now_Radiation'][j] = data['Radiation(today)(CWB)'][j]
        
        #next_Radiation_1(倒數4筆沒有下一小時的Radiation)
        if j in range(0,length_data_last):
            feature_data['next_Radiation_1'][j] = data['Radiation(today)(CWB)'][j+4]
        else:
            feature_data['next_Radiation_1'][j] = 0.0000 
    return feature_data

In [3]:
def Forecast_day_by_4_15_minutes():
    import datetime
    import pandas as pd
    from influxdb import InfluxDBClient

    # 使用 joblib.load 載入已訓練好的模型
    model = joblib.load('./model/4-15_minute/6-17/pred_power_m_6-17.pkl')

    # 取得新的預測資料
    new_data = pd.read_csv(f'Dataset/solar_汙水廠_merged(15m_ch_power).csv')

    # 只保留 'TIME_TO_INTERVAL' 在當下時間的上兩小時到下一個小時的資料（從當下時間的前兩小時到下一個小時）
    current_time = datetime.datetime.now()
    start_time = current_time - datetime.timedelta(hours=2)
    next_hour = current_time + datetime.timedelta(hours=1)
    new_data['TIME_TO_INTERVAL'] = pd.to_datetime(new_data['TIME_TO_INTERVAL'])
    new_data = new_data[(new_data['TIME_TO_INTERVAL'] >= start_time) & (new_data['TIME_TO_INTERVAL'] < next_hour)]

    # 重設索引
    new_data.reset_index(drop=True, inplace=True)
    # 補值操作
    new_data = new_data.interpolate(method='linear',limit_area=None)
    new_data = new_data.fillna(0)
    
    # 對新的資料進行前處理（類似於 split_data 函式）
    pre_data = split_data(new_data)

    # 使用已訓練好的模型進行預測
    prediction = model.predict(pre_data)

    # 將預測結果新增為一個欄位
    new_data['Prediction'] = prediction

    # 只保留 'TIME_TO_INTERVAL' 和 'Prediction' 欄位
    new_data = new_data[['TIME_TO_INTERVAL', 'Prediction']]

    # 取得當前時間的小時部分
    current_hour = current_time.hour

    # 過濾資料，只保留當前小時的整點到 45 分的資料
    filtered_data = new_data[
        (new_data['TIME_TO_INTERVAL'].dt.hour == current_hour) & 
        (new_data['TIME_TO_INTERVAL'].dt.minute >= 0) & 
        (new_data['TIME_TO_INTERVAL'].dt.minute <= 45)
    ]

    # 輸出選取的資料
    filtered_data

    #--------------------------------------------------------------------------------
    # 送資料到資料庫

    # 建立 InfluxDBClient
    client = InfluxDBClient('120.107.146.56', 8086, 'ncue01', 'Q!A@Z#WSX','Minute_Ahead_Pred')

    # 檢查資料庫是否已存在
    exist = client.get_list_database()
    database_exists = False
    for db in exist:
        if db['name'] == '15_minute_prediction':
            database_exists = True
            break

    # 建立或選擇資料庫
    if not database_exists:
        client.create_database('15_minute_prediction')
    client.switch_database('15_minute_prediction')

    # 轉換預測資料格式
    influx_data = []
    for i in range(len(filtered_data)):
        prediction_value = filtered_data['Prediction'].values[i]
        #檢查是否為負數，若是負數則將其乘以 -1 後再寫入資料庫
        if prediction_value < 0:
            prediction_value *= -1  # 將負數乘以 -1
        influx_data.append(
            {
                "measurement": "汙水場",
                "tags": {
                    "UpdateTime": datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                },
                "time": filtered_data['TIME_TO_INTERVAL'].values[i].astype('M8[ms]').item().strftime('%Y-%m-%d %H:%M:%S'),
                "fields": {
                    "Power": prediction_value
                }
            }
        )

    # 寫入數據
    client.write_points(influx_data)
    print(current_time,"預測資料寫入資料庫")

# 排程，每小時的第３分時執行

In [ ]:
import schedule
import joblib
import time
import datetime

# 設定排程
def scheduled_task():
    current_minute = datetime.datetime.now().minute
    if current_minute == 3:
        Auto_merge_15m()

schedule.every().hour.at(":03").do(scheduled_task)

while True:
    # 執行排程的任務
    schedule.run_pending()
    time.sleep(1)

舊的檔案已刪除
2023-08-04 14:03:00.693421 新的15分鐘資料存到CSV檔案中
2023-08-04 14:03:00.693421 Power的值替換成15分鐘
2023-08-04 14:03:01.963679 預測資料寫入資料庫
舊的檔案已刪除
2023-08-04 15:03:00.264008 新的15分鐘資料存到CSV檔案中
2023-08-04 15:03:00.264008 Power的值替換成15分鐘
2023-08-04 15:03:00.983910 預測資料寫入資料庫
舊的檔案已刪除
2023-08-04 16:03:00.705080 新的15分鐘資料存到CSV檔案中
2023-08-04 16:03:00.705080 Power的值替換成15分鐘
2023-08-04 16:03:01.423645 預測資料寫入資料庫
舊的檔案已刪除
2023-08-04 17:03:00.295314 新的15分鐘資料存到CSV檔案中
2023-08-04 17:03:00.295314 Power的值替換成15分鐘
2023-08-04 17:03:01.013891 預測資料寫入資料庫
舊的檔案已刪除
2023-08-04 18:03:00.855506 新的15分鐘資料存到CSV檔案中
2023-08-04 18:03:00.855506 Power的值替換成15分鐘
2023-08-04 18:03:01.574078 預測資料寫入資料庫
舊的檔案已刪除
2023-08-04 19:03:00.594834 新的15分鐘資料存到CSV檔案中
2023-08-04 19:03:00.594834 Power的值替換成15分鐘
2023-08-04 19:03:01.313377 預測資料寫入資料庫
舊的檔案已刪除
2023-08-04 20:03:00.647253 新的15分鐘資料存到CSV檔案中
2023-08-04 20:03:00.647253 Power的值替換成15分鐘
2023-08-04 20:03:01.381446 預測資料寫入資料庫
舊的檔案已刪除
2023-08-04 21:03:00.257794 新的15分鐘資料存到CSV檔案中
2023-08-04 21:03:00.257794 Powe